# M4 NO FAULT

IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M4\NO_FAULT4


SETTAGGIO DELLA VARIABILE RELATIVA AL GUASTO

In [3]:
guasto=0

## RCOU
VARIABILE RCOU POTENZA MOTORI.

Andiamo a prendere una potenza media di 1350 per considerare il momento il cui inizia/finisce il volo

In [4]:
rcou_m4_nofault = pd.read_csv("RCOU.csv")
rcou_m4_nofault = rcou_m4_nofault.astype("float64")
rcou_m4_nofault = rcou_m4_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350
rcou_m4_nofault = rcou_m4_nofault[((rcou_m4_nofault['C9'] >= potenza) & (rcou_m4_nofault['C10'] >= potenza) & (rcou_m4_nofault['C11'] >= potenza) & (rcou_m4_nofault['C12'] >= potenza) & (rcou_m4_nofault['C13'] >= potenza) & (rcou_m4_nofault['C14']>= potenza))]
rcou_m4_nofault=rcou_m4_nofault.reset_index(drop=True)
display(rcou_m4_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,76367829.0,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0
1,76399898.0,1372.0,1427.0,1439.0,1358.0,1409.0,1391.0
2,76420523.0,1389.0,1437.0,1444.0,1382.0,1417.0,1411.0
3,76439487.0,1471.0,1375.0,1402.0,1448.0,1446.0,1403.0
4,76471372.0,1411.0,1475.0,1464.0,1423.0,1428.0,1459.0
...,...,...,...,...,...,...,...
5094,136813708.0,1657.0,1369.0,1584.0,1468.0,1512.0,1545.0
5095,136823888.0,1633.0,1404.0,1581.0,1472.0,1479.0,1575.0
5096,136833858.0,1603.0,1445.0,1592.0,1458.0,1455.0,1594.0
5097,136848913.0,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0


Andiamo a prendere i tempi di inizio e di fine, così da poter effettuare il trimming delle altre tabelle

In [5]:
min=rcou_m4_nofault['TimeUS'][0]
max=rcou_m4_nofault['TimeUS'][len(rcou_m4_nofault)-1]
print(max)
print(min)

136861088.0
76367829.0


## ATT & XKF
### XKF
XKF utilizzato per fare poi il confronto con i valori di roll, pitch e yaw presenti att

In [6]:
xkf1_m4_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m4_nofault = xkf1_m4_nofault.astype("float64")
xkf1_m4_nofault = xkf1_m4_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m4_nofault = xkf1_m4_nofault[((xkf1_m4_nofault['TimeUS'] >= min) & (xkf1_m4_nofault['TimeUS'] <= max	))]
xkf1_m4_nofault = xkf1_m4_nofault.reset_index(drop=True)
print(xkf1_m4_nofault)

           TimeUS  Roll  Pitch     Yaw
0      76375630.0  1.69  -1.08   16.30
1      76386278.0  1.67  -1.10   16.29
2      76397328.0  1.62  -1.13   16.29
3      76407963.0  1.54  -1.13   16.28
4      76418382.0  1.51  -1.14   16.29
...           ...   ...    ...     ...
5474  136811131.0 -1.72   0.04  355.34
5475  136821892.0 -1.71   0.04  355.34
5476  136831987.0 -1.74   0.05  355.34
5477  136846510.0 -1.83   0.11  355.33
5478  136859050.0 -1.73   0.04  355.36

[5479 rows x 4 columns]


### ATT
Andiamo a prendere dal csv relativo ad ATT i dati relativi alle variabili di roll, pitch e yaw

In [7]:
att_m4_nofault = pd.read_csv("ATT.csv")
att_m4_nofault = att_m4_nofault.astype("float64")
att_m4_nofault = att_m4_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m4_nofault = att_m4_nofault[((att_m4_nofault['TimeUS'] >= min) & (att_m4_nofault['TimeUS'] <= max	))]
att_m4_nofault=att_m4_nofault.reset_index(drop=True)
print(att_m4_nofault)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      76377698.0     0.18  1.69      0.14  -1.08   16.23   16.30
1      76388276.0     0.18  1.67      0.15  -1.10   16.23   16.29
2      76399032.0     0.17  1.62      0.15  -1.13   16.23   16.29
3      76409551.0     0.17  1.54      0.16  -1.13   16.23   16.28
4      76419805.0     0.17  1.51      0.16  -1.14   16.23   16.29
...           ...      ...   ...       ...    ...     ...     ...
5475  136812901.0     0.75 -1.72     -1.40   0.04  354.13  355.35
5476  136823075.0     0.79 -1.71     -1.40   0.04  354.13  355.35
5477  136833108.0     0.84 -1.74     -1.41   0.05  354.13  355.35
5478  136848053.0     0.87 -1.83     -1.42   0.11  354.13  355.34
5479  136860307.0     0.90 -1.73     -1.44   0.04  354.13  355.37

[5480 rows x 7 columns]


Tramite un ciclo for si va a vedere se i valori di XKF corrispondono con quelli di ATT in caso contrario fa una media tra i due

In [8]:
from numpy import empty

if len(att_m4_nofault)>len(xkf1_m4_nofault):
    to_add=att_m4_nofault[len(xkf1_m4_nofault):]
    att_m4_nofault=att_m4_nofault[:len(xkf1_m4_nofault)]

if len(xkf1_m4_nofault)>len(att_m4_nofault):
    to_add=xkf1_m4_nofault[len(att_m4_nofault):]

for idx,i in enumerate(att_m4_nofault['Roll']):
    if(xkf1_m4_nofault['Roll'][idx] != i):
        att_m4_nofault['Roll'][idx] = ((att_m4_nofault['Roll'][idx] + xkf1_m4_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m4_nofault['Pitch']):
    if(xkf1_m4_nofault['Pitch'][idx] != i):
        att_m4_nofault['Pitch'][idx] = ((att_m4_nofault['Pitch'][idx] + xkf1_m4_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m4_nofault['Yaw']):
    if(xkf1_m4_nofault['Yaw'][idx] != i):
        att_m4_nofault['Yaw'][idx] = ((att_m4_nofault['Yaw'][idx] + xkf1_m4_nofault['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m4_nofault=pd.concat([att_m4_nofault,to_add])

print(att_m4_nofault)

           TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw      Yaw
0      76377698.0     0.18  1.690      0.14 -1.080   16.23   16.300
1      76388276.0     0.18  1.670      0.15 -1.100   16.23   16.290
2      76399032.0     0.17  1.620      0.15 -1.130   16.23   16.290
3      76409551.0     0.17  1.540      0.16 -1.130   16.23   16.280
4      76419805.0     0.17  1.510      0.16 -1.140   16.23   16.290
...           ...      ...    ...       ...    ...     ...      ...
5475  136812901.0     0.75 -1.715     -1.40  0.040  354.13  355.345
5476  136823075.0     0.79 -1.725     -1.40  0.045  354.13  355.345
5477  136833108.0     0.84 -1.785     -1.41  0.080  354.13  355.340
5478  136848053.0     0.87 -1.780     -1.42  0.075  354.13  355.350
5479  136860307.0     0.90 -1.730     -1.44  0.040  354.13  355.370

[5480 rows x 7 columns]


## IMU
VALORI DI ACCELERAZIONE E DEL GIROSCOPIO

Per le 3 tabelle di IMU si sono presi i soli valori relativi a GyrX, GyrY, GyrZ, AccX, AccY, AccZ.
In seguito si è poi creata una singola tabella popolandola con la media dei valori delle tre tabelle.

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m4_nofault = pd.read_csv("IMU_0.csv")
imu_0_m4_nofault = imu_0_m4_nofault.astype("float64")
imu_0_m4_nofault = imu_0_m4_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m4_nofault = imu_0_m4_nofault[((imu_0_m4_nofault['TimeUS'] >= min) & (imu_0_m4_nofault['TimeUS'] <= max))]

imu_1_m4_nofault = pd.read_csv("IMU_1.csv")
imu_1_m4_nofault = imu_1_m4_nofault.astype("float64")
imu_1_m4_nofault = imu_1_m4_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m4_nofault = imu_1_m4_nofault[((imu_1_m4_nofault['TimeUS'] >= min) & (imu_1_m4_nofault['TimeUS'] <= max))]

imu_2_m4_nofault = pd.read_csv("IMU_2.csv")
imu_2_m4_nofault = imu_2_m4_nofault.astype("float64")
imu_2_m4_nofault = imu_2_m4_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m4_nofault = imu_2_m4_nofault[((imu_2_m4_nofault['TimeUS'] >= min) & (imu_2_m4_nofault['TimeUS'] <= max))]

imu_m4_nofault = pd.concat((imu_0_m4_nofault, imu_1_m4_nofault, imu_2_m4_nofault))
imu_m4_nofault=imu_m4_nofault.groupby(imu_m4_nofault.TimeUS, as_index=False).mean()

display(imu_m4_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,76369982.0,-0.200508,-0.064201,-0.024340,-0.141876,-0.301092,-9.360346
1,76372073.0,-0.176018,-0.085531,-0.030017,-0.164276,-0.126333,-9.419593
2,76374333.0,0.026623,-0.026614,-0.010593,-0.314321,0.048481,-9.672186
3,76377446.0,0.168383,0.042279,0.012229,-0.332445,0.063955,-9.758467
4,76380512.0,0.025879,0.046393,0.026676,-0.255138,-0.149288,-9.542381
...,...,...,...,...,...,...,...
20834,136835711.0,0.067217,-0.045699,-0.004597,-0.369982,0.197985,-9.371787
20835,136841561.0,-0.131265,0.172866,0.017041,-0.259921,0.339389,-9.677052
20836,136847878.0,0.088613,-0.040721,-0.016431,-0.189151,0.475869,-10.086913
20837,136853859.0,-0.044895,0.002809,0.021365,0.192493,1.075484,-10.305997


## UNIONE DELLE TABELLE
Vengono unite le tabelle relative a IMU, ATT e RCOU, riempiendo i valori mancanti con quelli vicini

In [11]:
m4_nofault = pd.merge_ordered(imu_m4_nofault,att_m4_nofault)
m4_nofault=m4_nofault.fillna(method="ffill").fillna(method="bfill")
m4_nofault = pd.merge_ordered(m4_nofault, rcou_m4_nofault)
m4_nofault=m4_nofault.fillna(method="ffill").fillna(method="bfill")

Vengono aggiornati i tempi facendo partire il volo all'istante 0, e viene inserita la colonna relativa al guasto, valorizzandola con la variabile *guasto*.

In [12]:
m4_nofault["TimeUS"] = m4_nofault["TimeUS"] - m4_nofault.iloc[0]["TimeUS"]
m4_nofault["TimeUS"] = m4_nofault["TimeUS"].astype(int)
m4_nofault["Guasto"] = guasto

In [ ]:
last_cell = m4_nofault.iloc[-1, m4_nofault.columns.get_loc('TimeUS')]
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)
df = pd.DataFrame(to_be, columns = ['TimeUS'])

Viene corretta la frequenza di campionamento in modo tale da avere 350 misurazioni per ogni secondo

In [ ]:
m4_nofault_final = pd.merge_ordered(m4_nofault,df)
m4_nofault_final=m4_nofault_final.fillna(method="ffill").fillna(method="bfill")
m4_nofault_final = m4_nofault_final[m4_nofault_final.TimeUS.isin(to_be)]

Viene formattata la colonna del tempo in maniera tale che sia più leggibile all'operatore umano

In [19]:
from datetime import timedelta
m4_nofault_final=m4_nofault_final.reset_index(drop=True)
m4_nofault_final['TimeUS'] = pd.to_datetime(m4_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m4_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.200508,-0.064201,-0.024340,-0.141876,-0.301092,-9.360346,0.18,1.690,0.14,-1.080,16.23,16.30,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
1,00:00:00.002857,-0.200508,-0.064201,-0.024340,-0.141876,-0.301092,-9.360346,0.18,1.690,0.14,-1.080,16.23,16.30,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
2,00:00:00.005714,-0.176018,-0.085531,-0.030017,-0.164276,-0.126333,-9.419593,0.18,1.690,0.14,-1.080,16.23,16.30,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
3,00:00:00.008571,0.026623,-0.026614,-0.010593,-0.314321,0.048481,-9.672186,0.18,1.690,0.14,-1.080,16.23,16.30,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
4,00:00:00.011428,0.168383,0.042279,0.012229,-0.332445,0.063955,-9.758467,0.18,1.690,0.14,-1.080,16.23,16.30,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21169,00:01:00.479833,-0.131265,0.172866,0.017041,-0.259921,0.339389,-9.677052,0.84,-1.785,-1.41,0.080,354.13,355.34,1603.0,1445.0,1592.0,1458.0,1455.0,1594.0,0.0
21170,00:01:00.482690,0.088613,-0.040721,-0.016431,-0.189151,0.475869,-10.086913,0.87,-1.780,-1.42,0.075,354.13,355.35,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0,0.0
21171,00:01:00.485547,0.088613,-0.040721,-0.016431,-0.189151,0.475869,-10.086913,0.87,-1.780,-1.42,0.075,354.13,355.35,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0,0.0
21172,00:01:00.488404,-0.044895,0.002809,0.021365,0.192493,1.075484,-10.305997,0.87,-1.780,-1.42,0.075,354.13,355.35,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0,0.0


## Analisi nel tempo e in frequenza

Funzione che calcola il Root Mean Square

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

Viene definita la funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m4_nofault_final)/350)):
        V1=m4_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

Si richiama la funzione descritta sopra per ogni variabile in modo da avere le statistiche sia nel tempo che in frequenza

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')

merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_13860\3233662715.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_13860\3233662715.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.044203,0.010318,0.334582,0.110645,75.0,0.0,4.446807,4.385723,0.043998,0.004702,-0.520986,0.081389,101.0,100.0,2.826721,1.406813,0.100953,0.006508,-1.509005,0.129155,73.0,62.0,0.898254,0.669778,-0.069917,0.069479,-0.345449,0.272341,106.0,110.0,9.033675,7.518019,0.195170,0.071388,-0.593695,0.330570,99.0,106.0,7.885252,7.596394,-10.902930,0.727518,-1.259503,10.936148,0.0,115.0,153.126164,27.069541,0.928148,1.171023,-0.814566,1.424481,87.0,86.0,24.801296,6.157553,-0.421453,0.451763,0.515405,0.792527,0.0,82.0,42.913828,2.793667,18.371168,4.531613,-1.360700,18.493743,108.0,79.0,-0.242574,-1.262065,1598.051282,8528.460220,-0.256391,1600.709862,91.0,90.0,1146.929677,646.140850,1517.780627,3618.537452,1.321226,1518.968814,84.0,82.0,1395.355831,597.844465,1584.313390,7952.192934,-0.193914,1586.813932,89.0,88.0,779.137078,497.907756,1533.316239,3792.176850,0.817250,1534.548814,99.0,97.0,1310.914690,443.470499,1557.048433,4949.469076,-0.261309,1558.632475,119.0,118.0,1068.833677,823.455498,1561.928775,6109.643484,0.167225,1563.877788,123.0,120.0,497.445609,437.949006,0
1,1.0,-0.044190,0.010288,0.343789,0.110508,66.0,0.0,4.601441,4.522906,0.044342,0.004730,-0.531587,0.081748,94.0,93.0,2.868816,1.379779,0.100829,0.006495,-1.507238,0.129007,79.0,68.0,0.936487,0.533439,-0.068669,0.069830,-0.366124,0.272666,112.0,116.0,10.439269,10.038134,0.195392,0.071202,-0.588479,0.330421,96.0,103.0,8.847203,7.380745,-10.901352,0.726322,-1.258643,10.934520,0.0,123.0,154.522104,26.984099,0.922415,1.179257,-0.835425,1.423642,94.0,93.0,25.230817,6.023464,-0.416165,0.460320,0.502938,0.795113,0.0,88.0,43.846063,2.613423,18.381733,4.557991,-1.367843,18.504949,117.0,87.0,-0.208626,-1.420471,1598.286932,8523.709458,-0.248571,1600.943663,85.0,82.0,1145.966496,644.511764,1517.676136,3612.071452,1.315122,1518.862293,89.0,88.0,1500.386820,554.687174,1584.494318,7941.059797,-0.184950,1586.991098,85.0,84.0,752.346332,541.422521,1533.286932,3781.675270,0.825346,1534.516128,105.0,103.0,1264.965671,447.247235,1557.125000,4937.431624,-0.252128,1558.705126,117.0,116.0,1076.231850,846.106556,1562.014205,6094.806065,0.178974,1563.958397,116.0,119.0,582.991270,387.817810,0
2,2.0,-0.044136,0.010260,0.351586,0.110359,0.0,72.0,4.673353,4.422251,0.044639,0.004748,-0.542972,0.082018,93.0,92.0,2.882469,1.322950,0.100673,0.006485,-1.506592,0.128847,81.0,80.0,0.965180,0.342812,-0.067270,0.070322,-0.385291,0.273218,110.0,106.0,12.257394,10.903195,0.195840,0.071071,-0.587150,0.330489,97.0,99.0,9.445986,7.841521,-10.899565,0.725385,-1.258691,10.932697,0.0,117.0,155.979498,26.945405,0.916629,1.187724,-0.855388,1.422873,89.0,88.0,25.604412,5.850641,-0.410595,0.469964,0.507273,0.798261,0.0,92.0,44.874354,2.515427,18.392351,4.584842,-1.374557,18.516221,112.0,82

## Output
Salviamo la tabella ottenuta in un file csv

In [23]:
path_file = path_file.replace(r"M4\NO_FAULT4", "")
os.chdir(path_file)
df_merged.to_csv('m4_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
